In [1]:
!pip install gym

     |████████████████████████████████| 721 kB 3.2 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827632 sha256=d9b2d87b9ec2ecfb3446c59d26de14cc93cc1be119399feb0c123eaf3491c42d
  Stored in directory: /Users/rish/Library/Caches/pip/wheels/af/2b/30/5e78b8b9599f2a2286a582b8da80594f654bf0e18d825a4405
Successfully built gym


# Set up Environment

In [2]:
import gym 
import random

In [10]:
env = gym.make("CartPole-v1", render_mode="rgb_array")
states = env.observation_space.shape[0]
actions = env.action_space.n

In [11]:
actions # Two directions of movement 

2

In [12]:
episodes = 10 

for episode in range(1, episodes + 1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score += reward
        print(episode, score)

DependencyNotInstalled: pygame is not installed, run `pip install gym[classic_control]`